<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/_notebook/Testing_GradientMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.cs.cmu.edu/~pradeepr/convexopt/Lecture_Slides/dual-ascent.pdf

In [61]:
import numpy as np
# import torch
import matplotlib.pyplot as plt

In [88]:
# dim=2000000
# a = 100*np.random.randn(4, dim) + 1123
# g = np.random.randn(dim, 1)
# b = (a-np.mean(a, axis=1)[:, np.newaxis])/np.std(a, axis=1)[:, np.newaxis]
# np.mean(b@g/dim)

In [63]:
def gradient_maker(domain_grads, epsilon=0.5, alpha=0.01, eff=0.1):
    def log():
        print(f"Iter={iter}, Condition={np.min(dgr@g/dim)}")
    
    # def null(A, eps=1e-15):
    #     u, s, vh = np.linalg.svd(A)
    #     # print(vh)
    #     null_space = np.compress(s <= eps, vh, axis=0)
    #     return null_space.T

    def null(A):
        from scipy.linalg import null_space
        n = null_space(A, rcond=1)
        if n.shape[1] < 1:
            return False, None
        r = np.sqrt(np.sum((A@n)**2, axis=0)/n.shape[1])
        if np.min(r) >= 1e-2:
            return False, None
        return True, n.T[np.argmin(r)].reshape(-1, 1)
    
    def scale():
        G = np.concatenate((g.reshape(1, -1), domain_grads), axis=0)
        is_fine, n = null(G@G.T)
        if not is_fine:
            return np.mean(np.sqrt(np.sum(domain_grads**2, axis=1)))/np.sqrt(np.sum(g**2))
        n /= np.sum(n[1:])
        return -1.*n[0,0]

    dgr = np.copy(domain_grads)
    number_of_domains, dim = dgr.shape

    #
    dgr /= np.sqrt(np.sum(dgr**2, axis=1))[:, np.newaxis]
    
    #
    g = np.random.randn(dim)
    g /= np.sqrt(np.sum(g**2))
    u_ = np.zeros(number_of_domains)
    
    iter = 0
    while not np.min(dgr@g/dim) >= epsilon:

        log()
        
        u_ = u_ + alpha*((1.+eff)*epsilon - (dgr@g)/dim)
        g = (1./number_of_domains)*np.sum(((1+u_).reshape(number_of_domains, 1))*dgr, axis=0)

        iter += 1
    
    log()
    return scale()*g

### Testcases

Now let's write some test cases to make sure everything is working correctly


In [64]:
epsilon=10
alpha=0.9

In [65]:
grads = 20*np.random.randn(2, 3) - 10
print(grads)

[[-51.52835826   2.69042404 -29.05399967]
 [-19.75661257 -22.04804061  -8.63253508]]


In [66]:
g = gradient_maker(grads, epsilon=epsilon, alpha=alpha)
print(g)

Iter=0, Condition=0.006419617094602832
Iter=1, Condition=2.9936519066332488
Iter=2, Condition=4.989232803740066
Iter=3, Condition=6.487250990448548
Iter=4, Condition=7.611773241104378
Iter=5, Condition=8.45592997922697
Iter=6, Condition=9.089629352937443
Iter=7, Condition=9.565347780570612
Iter=8, Condition=9.9224766049462
Iter=9, Condition=10.190584895343134
[-30.61740344 -13.59149265 -15.61336825]
